In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# !pip install datatable > /dev/null
# import datatable as dt
from tqdm import tqdm
from sklearn import model_selection
from sklearn import metrics
import time

import tensorflow as tf
from tensorflow import keras
import janestreet


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
sample_submission = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
test_df = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
# features_df = pd.read_csv('../input/jane-street-market-prediction/features.csv')
#train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
# train_df = train_data_datatable.to_pandas()
train_df = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train_df = train_df[train_df.weight != 0]
train_df['action'] = (train_df['resp'].values > 0).astype(int)

In [ ]:
print(train_df.shape)
train_df.head()

In [ ]:
for col in list(train_df.columns):
    print(col, ' : ', train_df[col].nunique())

In [ ]:
train_df.info()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 30))

ax[0, 0].set_xlim(40)
ax[0, 0].hist(train_df['feature_111'])
ax[1, 0].hist(train_df['feature_112'])
ax[2, 0].hist(train_df['feature_113'])

ax[0, 1].hist(train_df['feature_120'])
ax[1, 1].hist(train_df['feature_121'])
ax[2, 1].hist(train_df['feature_122'])
plt.show()

In [ ]:
plt.figure(figsize = (10, 5))
a = train_df.groupby(['date'])['feature_10'].sum()
plt.plot(a.index, a.values)
# plt.plot(train_df.groupby(['date'])['feature_11'].sum())
# plt.plot(train_df.groupby(['date'])['feature_12'].sum())
# plt.plot(train_df.groupby(['date'])['feature_13'].sum())
# plt.plot(train_df.groupby(['date'])['feature_14'].sum())
# plt.plot(train_df.groupby(['date'])['feature_15'].sum())
# plt.plot(train_df.groupby(['date'])['feature_16'].sum())
plt.show()

### Missing Values

In [ ]:
vars_with_na = [var for var in train_df.columns if train_df[var].isnull().mean() > 0]
train_df[vars_with_na].isnull().mean().sort_values(ascending = False)

In [ ]:
# data_na = train_df[vars_with_na].isnull().mean()*100
# data_na = pd.DataFrame(data_na.reset_index())
# data_na.columns = ['variable', 'percentage']
# data_na.sort_values(by = 'percentage', ascending = False, inplace = True)
# print('There are features which are not missing at random. So some event might have taken place for the common missing values')
# data_na[data_na.percentage > 10]

In [ ]:
# plt.figure(figsize = (200, 10))
# plt.plot(train_df.date, train_df.feature_18.isnull())

This is not completely missing at random.
Now I will look along with the date if it had something to do with the date

In [ ]:
# Fill Missing value with the Mean or Median

def impute_na(df, variable, mean_median):
    return df[variable].fillna(mean_median)

train_df_imputed = train_df.copy()

for var in vars_with_na:
    mean = train_df_imputed.loc[:, var].mean()
    train_df_imputed[var] = impute_na(train_df_imputed, var, mean)

train_df_imputed.isnull().mean().sort_values(ascending = False)

In [ ]:
# # There is a very little change in the mean
# train_df['feature_18'].mean(), train_df_imputed['feature_18'].mean()

In [ ]:
# figure = plt.figure(figsize = (50, 5))
# ax = figure.add_subplot(211)
# ax2= figure.add_subplot(212)
# train_df.feature_18.plot(kind = 'kde', ax = ax, color = 'blue', label = 'Missing values')
# train_df_imputed.feature_18.plot(kind = 'kde', ax = ax,color = 'red', label = 'Imputed')

# train_df.feature_28.plot(kind = 'kde', ax = ax2, color = 'blue', label = 'Missing values')
# train_df_imputed.feature_28.plot(kind = 'kde', ax = ax2,color = 'red', label = 'Imputed')
# plt.legend()
# plt.show()

In [ ]:
# There is a very little change in variance
a = (train_df.feature_18.var(), train_df_imputed.feature_18.var())
print(f'Before: {a[0]} \nAfter : {a[1]} \n% Change in variance: {(a[0] - a[1]) * 100/a[0]}')

In [ ]:
# There is a very little change in Standard Deviation
a = (train_df.feature_102.std(), train_df_imputed.feature_102.std())
print(f'Before: {a[0]} \nAfter : {a[1]} \n%Change in std: {(a[0] - a[1]) * 100/a[0]}')

In [ ]:
# train_df_imputed.feature_102.plot(kind = 'box', grid = True, figsize = (10,5))

In [ ]:
# len(train_df_imputed[train_df_imputed.feature_102.sort_values(ascending=  False) > 3]) * 100 / len(train_df_imputed)

In [ ]:
# train_df.cov()

In [ ]:
# train_df.corr()

In [ ]:
print(train_df.weight.min(), train_df.weight.max())
train_df.weight.plot(kind = 'hist')

In [ ]:
# Which features we can drop the train and test sets.
features = [col for col in test_df.columns if col in train_df.columns]
target = 'action'
features.remove('ts_id')
features

In [ ]:
# X_train = train_df_imputed.loc[:, features]
# y_train = train_df_imputed.loc[:, 'action']

del train_df

## Using KFold Technique for cross validation

In [ ]:
train_df_imputed = train_df_imputed.loc[:, features + [target]]
train_df_imputed = train_df_imputed.sample(frac = 1).reset_index(drop = True)

train_df_imputed['kfold'] = -1

splits = 2
epochs = 1

kf = model_selection.KFold(n_splits = splits)

for fold, (trn_, val_) in enumerate(kf.split(X = train_df_imputed)):
    train_df_imputed.loc[val_, 'kfold'] = fold

# train_df_imputed.to_csv('train_folds.csv', index = False)

In [ ]:
train_df_imputed.kfold.unique()

In [ ]:
def run(fold, clf, epochs,nn = False):
    df_train = train_df_imputed[train_df_imputed.kfold != fold].reset_index(drop = True)
    df_valid = train_df_imputed[train_df_imputed.kfold == fold].reset_index(drop = True)
    
    print('Train Shape', df_train.shape)
    print('Valid Shape', df_valid.shape)
    
    x_train = df_train.drop(['action', 'kfold'], axis = 1).values
    y_train = df_train.action.values
    
    x_valid = df_valid.drop(['action', 'kfold'], axis = 1).values
    y_valid = df_valid.action.values
    
    if not nn:
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_valid)
        score = metrics.roc_auc_score(y_true, y_pred)
        logloss = metrics.log_loss(y_true, y_pred)
        print(f'Running:{fold} iteration, Accuracy = {score * 100}%, log_Loss = {logloss}')
    
    if nn:
        print(f'Currently running: {fold}')
        history = clf.fit(x_train, y_train, epochs = epochs, validation_data = (x_valid, y_valid))    
        return clf, history

In [ ]:
learning_rate = 1e-5
label_smoothing = 1e-4

model = keras.models.Sequential()
#model.add(keras.layers.Flatten(input_shape = [1, 139]))
model.add(keras.layers.Dense(300, activation = 'relu'))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(1, activation = 'sigmoid'))

model.compile(
             optimizer =tf.keras.optimizers.Adam(learning_rate = learning_rate),
             loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing),
             metrics = tf.keras.metrics.AUC(name = 'AUC'))
histories = []
for val in range(splits):
    model, history = run(fold = val, clf = model, epochs = epochs, nn = True)
    histories.append(history)

In [ ]:
print(model.layers)

In [ ]:
model.summary()

In [ ]:
f_mean = train_df_imputed[features].mean()
train_df_imputed = train_df_imputed.query('weight > 0').reset_index(drop = True)
print(f_mean)

In [ ]:
janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()

start_time = time.time()
opt_th = 0.5
for test_df, pred_df in env.iter_test():
    pred_df.action = 0
    if test_df['weight'].item() > 0:
        # Handling Missing Values        
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt.sum()):
            pred_df.action = 0
        else:   
            pred = model(x_tt, training = False).numpy().item()
            pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
        
    env.predict(pred_df)

print(f'Finished Processing. It took {time.time() - start_time} seconds')